Cell for working not locally

Make git clone if we aren't in the right directory

In [2]:
import subprocess
import os

repo_path = "YAGPT"
branch = "dev"

if not os.path.isdir(repo_path):
    subprocess.run(["git", "clone", "https://github.com/RodionfromHSE/YAGPT.git"])
    
os.chdir(repo_path)
subprocess.run(["git", "checkout", branch])

CompletedProcess(args=['git', 'checkout', 'dev'], returncode=0)

Windows users

In [3]:
import json
from pprint import pprint

params = json.load(open('config.json'))
onServer = True
if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['history: ', 'speaker1: ', 'speaker2: '],
 'batch_size': 8,
 'block_size': 128,
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'dataset_name': 'under-tree/prepared-yagpt',
 'num_proc': 8}


In [4]:
%%capture
!pip install transformers datasets huggingface_hub

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from datasets import load_dataset
from pprint import pprint

name = params['dataset_name']
dataset = load_dataset(name)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print('len', len(dataset))
pprint(dataset['train'][0])
pprint(dataset['train'][-7])

len 2
{'text': 'history: \n'
         'speaker1: как земные географы относятся к идее освоения и '
         'колонизации луны и марса?\tspeaker2: к планам освоения луны отношусь '
         'с большой осторожностью. \n'
         'есть одна грань, о которой у нас почти не говорят. на луне очень '
         'страшная пыль. дело в том, что эта пыль размером в 2-3 микрона. '
         'каждая пылинка имеет зазубренные концы. попадая в легкие, такая пыль '
         'вызывает страшную болезнь – лунную сенную лихорадку. все 15 '
         'астронавтов из экспедиции космического корабля "аполлон", среди '
         'которых были и те, которые не выходили на поверхность луны, '
         'переболели этой болезнью. скафандры до пояса были покрыты пылью. она '
         'попадала на всех участников экспедиции. пока ученые не знают, как '
         'избавиться от лунной пыли.\n'
         'проблема с освоением марса, даже если хватит денег туда долететь и '
         'там поработать, связана с возможностью 

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def tokenize(elem):
    inputs = tokenizer(elem['text'])
    return inputs

data_token = dataset.map(tokenize, batched=True, remove_columns=dataset['train'].column_names, num_proc=params['num_proc'])

In [10]:
block_size = params['block_size']

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

data_prep = data_token.map(group_texts, batched=True, num_proc=params['num_proc'])

In [14]:
for (key, value) in data_prep['train'][0].items():
  print(f'key {key}, value len {len(value)}, value type {type(value)}')

key input_ids, value len 128, value type <class 'list'>
key attention_mask, value len 128, value type <class 'list'>
key labels, value len 128, value type <class 'list'>


In [18]:
del training_args

In [41]:
!nvidia-smi

Sun Apr  9 18:01:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

2301

In [46]:
!nvidia-smi

Sun Apr  9 18:03:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [47]:
from transformers import Trainer, TrainingArguments

batch_size = params['batch_size']
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data_prep['train'],
    eval_dataset=data_prep['test']
)

In [48]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)